`api` `pandas` `stocks` `yfinance` `threading` `csv` `s&p 500` `stock market index calculating`

Ссылка на этот листок:

https://colab.research.google.com/drive/1w9O6rSj36jsvgw-WIO9TJHMY4eRh9tEU?usp=sharing

http://bit.ly/US10F_index

Полезные ссылки:
https://pypi.org/project/yfinance/

https://stackoverflow.com/questions/58702437/python-yahoo-finance-error-market-cap-intdata-get-quote-yahoostrmarketcap

https://quant.stackexchange.com/questions/1640/where-to-download-list-of-all-common-stocks-traded-on-nyse-nasdaq-and-amex

ftp://ftp.nasdaqtrader.com/symboldirectory/

In [ ]:
!python -m pip install pandas_datareader

### Матчасть
Тикер -- короткий уникальный идентификатор (1-6 символов) акции (и не только). Нужен, чтобы не писать полностью названия компании.

Например:
- тикер Microsoft -- MSFT
- Apple -- AAPL
- AT&T -- T
- Северсталь -- CHMF (потому что владеет Череповецким (CHerepovets) металлургическим (Metal) комбинатом (Factory))
- Газпром -- GAZP
- Газпром Нефть -- SIBN (кто догадается, почему так?)

**MC** -- _market cap(italization)_ -- рыночная капитализация компании.

В самом простом случае считается как количество акций компании, умноженное на стоимость одной акции.
(но эта информация уже будет вам доступна, считать ничего не нужно)

**Фондовый индекс** (stock_market_index) -- если простыми словами, то множество каких-то ценных бумаг (в нашем случае акций), каждая из которых взята с каким-то весом (сумма весов всех взятых в индекс компаний -- разумеется, 1).

В зависимости от принципа, положенного в основу выбора ценных бумаг для индекса, он может отражать ценовую динамику группы ценных бумаг, объединённых по какому-то признаку (к примеру высокая, средняя, малая капитализация акций) выбранного сектора рынка (к примеру, телекоммуникации), или широкого рынка акций в целом.

Пример простого (и не очень осмысленного) индекса:
```
[
    ("Apple Inc.", "50.00%"),
    ("Visa Inc.", "40.00%"),
    ("Nike, Inc.", "10.00%")
]
```
Индекс может быть:
- взвешенным по цене (Dow Jones, Nikkei 225) -- вес каждой компании пропорционален цене её акции. Странно, но вот так бывает.
- взвешенным по рыночной капитализации (S&P 500, ММВБ) -- вес каждой компании пропорционален её рыночной капитализации. Такой индекс мы и будем составлять :)
- равновзвешенным (S&P 500 Equal Weighted, ÖkoDAX) -- вес каждой компании одинаков. 

In [ ]:
def get_tickers_list():
    # в этой функции вам нужно как-то скачать список тикеров из файлов
    # ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt
    # ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqlisted.txt
    # вам может пригодиться urllib.request.urlopen(), .decode("utf-8")
    

def download_companies_data(tickers_list):
    """
    tickers_list -- список тикеров. Например, ["MSFT", "AAPL", "TSM", ...]
    Функция должна с помощью библиотеки yfinance добыть данные о всех компаниях по их тикерам.
    Данные, которые вам нужны -- это те данные, которые понадобятся в build_US10F_index(), то есть:
    - короткое имя компании (например, "Apple Inc.")
    - MC компании (например, 13238984)
    - страна компании (например, "United States")

    Помните, что тикеров у вас в сумме где-то 9000!
    И загружать информацию по всем занимает несколько часов!!!
    Поэтому есть совет: аккуратно сохранять в файл то, что уже загружено.
    А то обидно будет всё начинать с нуля!
    """

### Полезные советы

Допустим, вы хотите скачать всю информацию не за один раз/у вас прервётся интернет/произойдёт что-то ещё.
В этом случае, скорее всего, вы будете запускать какую-то функцию несколько раз.

Есть РИСК, что вы пишете в файл, перезапишете его, и потеряете всю уже скачанную информацию.

Как избежать этого риска?
1. открывать файл в режиме "a" -- чтобы файл не перезаписывать, а дописывать в конец.
```
with open(file_name, "a"... ) as ...:
                 не "w"! ^
```
2. перед началом работы кода функции делать бекап файла, который уже существует.
```
import shutil
from datetime import datetime
def f(…, out_file_name, …):
        shutil.copy(out_file_name,
                    f"{out_file_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
        # ваш код
```
3. придумать что-то своё.

Может случиться так, что по какому-то тикеру информации нет, и вам кинется ошибка. В этом случае НЕ ХОТЕЛОСЬ БЫ
- терять уже скачанную информацию (хорошо бы сохранить её в файл)
- завершать программу (лучше вывести информацию, что с этим тикером всё грустно, и перейти к следующему).

Давайте напишем пример такой функции:


In [ ]:
# Пусть функция save_some_data() занимается сохранением данных в файл

def f(tickers_list, *, cache_file_name=None, ...):
    tickers_info = {}
    if cache_file_name:
        # достаём уже скачанные тикеры и отправляем их в tickers_info
    for ticker in tickers_list:
        # можно добавить принудительное сохранение после каждых N тикеров
        if ticker in tickers_info:
            continue  # инфо по этому тикеру уже была скачАна
        try:
            print(f"Processing {ticker} ticker")
            # какой-то код, заполняющий tickers_info
        except KeyboardInterrupt as exception:
            # мы можем хотеть завершить выполнение функции вручную (Ctrl+C в Linux/Mac, остановить блок в колабе)
            print("Processing aborted!")
            # сохраняем то, что успели скачать
            save_some_data(tickers_info)
            # и после этого уже завершаем выполнение программы, кидая ошибку
            raise exception
        except BaseException as exception:
            # ловим все остальные ошибки
            print(f"{ticker}: {type(exception)}")
            print(str(exception))
            # сохраняем в tickers_info информацию о тикере (что мы ничего не получили)
            # продолжаем работу!
            continue
    # Пишем, что закончили, сохраняем данные.
    print("Processing finished")
    save_some_data(tickers_info)


In [ ]:
def build_US10F_index(tickers_data, *,
                      num_companies: int,
                      min_market_cap: int = 0,
                      max_weight: float = 1.00):
    """
    tickers_data -- список информации об акциях, из которых мы собираем индекс.
    Выглядит он следующим образом:
    [
        {
            "ticker": "AAPL",
            "short_name: "Apple Inc.",
            "market_cap": 2269000000000,
            "country": "United States"
        },
        {
            "ticker": "XOM",
            ...
        }
    ]
    num_companies -- количество компаний, которое должно быть в индексе.
    min_market_cap -- минимальная рыночная капитализация, при которой компания может войти в индекс.
    max_weight -- число из (0; 1] -- максимальный вес, с которым компания может войти в индекс.
    
    Функция должна
    - выбрать из tickers_data акции американских компаний;
    !!! СОВЕТ! Если вы используете pandas_datareader, подумайте хорошо,
    как отличить американские компании от неамериканских.
    - выбрать из них num_companies компаний с наибольшей MC;
    - вернуть индекс: упорядоченный по убыванию MC список пар (short_name компании, её вес),
    где вес компании равен отношению MC компании к суммарной MC всех компаний, входящих в индекс.
    Если вес компании больше max_weight, она должна быть включена с весом max_weight.
    В этом случае оставшийся вес распределяется на все оставшиеся компании.
    Вес -- не число, а строка -- процент, округлённый до 2 знаков после запятой. Например, "4.25%".
    Имя -- нужно взять из поля "short_name".
    Размер списка -- разумеется, num_companies.

    Например:
    [
        ["Apple Inc.", "6.14%"],
        ["Microsoft Corporation", "4.88%"],
        ...
    ] (500 компаний)
    [
        ["Apple Inc.", "9.12%"],
        ["Microsoft Corporation", "7.24%"],
        ...
    ] (100 компаний)
    """
    if num_companies is None:
        raise TypeError("Specify number of companies in index")
    # Ваш код

In [ ]:
!python -m pip install yfinance  # медленная библиотека
!python -m pip install pandas_datareader  # быстрая библиотека

In [ ]:
import pandas_datareader
import yfinance
tickers = ["AAPL", "MSFT", "TSLA"]

In [ ]:
%%timeit
# пример, как получить данные через yfinance (долго)
for ticker in tickers:
    info = yfinance.Ticker(ticker).info
    # print(info)
print("OK")

In [ ]:
%%timeit
# пример, как получить данные через pandas_datareader (гораздо быстрее)
for ticker in tickers:
    info = pandas_datareader.get_quote_yahoo(ticker)
    print(dict(info))
    # print(info)
print("OK")